#### 1. Load the dataset, airfoil_self_noise.DAT, into memory.

In [1]:
import pandas as pd
import numpy as np
columns = ['frequency', 'attack_angle', 'chord_length', 'free_stream_velocity',
           'suction_side_disp', 'scaled_sound_pressure']
df = pd.read_csv('airfoil_self_noise.dat', sep='\t', header=None)
df.columns = columns
df.head()

,frequency,attack_angle,chord_length,free_stream_velocity,suction_side_disp,scaled_sound_pressure
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


#### 2. Clean the data and check missing values for this dataset.

In [2]:
df.isnull().sum(axis=0)

frequency                0
attack_angle             0
chord_length             0
free_stream_velocity     0
suction_side_disp        0
scaled_sound_pressure    0
dtype: int64

#### 3.a Split the data into 80% of training and 20% of the test dataset

In [3]:
X = df.drop('scaled_sound_pressure', axis=1)
y = df.scaled_sound_pressure

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2021)

#### 3.b Build a simple linear regression to forecast "Scaled sound pressure level"

In [4]:
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error

lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

print(f'''
The mean square error is {np.round(mean_squared_error(y_test,pred),4)}
The root mean square error is {np.round(mean_squared_error(y_test,pred, squared = False),4)}
The mean absolute error is {np.round(mean_absolute_error(y_test,pred),4)}
''')


The mean square error is 25.0018
The root mean square error is 5.0002
The mean absolute error is 3.8455



#### 4. Preprocess the data using the normalization

In [5]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
X_train= normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

#### 5. Build a deep learning regression model to forecast "Scaled sound pressure level" using all other features and TensorFlow

In [6]:
import tensorflow as tf

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
# Define Sequential model with  layers
model = keras.Sequential(
    [
        layers.Dense(20, activation='relu'),
        layers.Dense(1),
    ]
)

In [9]:
model.compile(optimizer='sgd',loss='mse')

In [10]:
tf.random.set_seed(1)
model.fit(x=X_train,y=y_train,batch_size=64,epochs=100,
          validation_data=(X_test,y_test)
          )

Epoch 1/100
19/19 [==============================] - 1s 18ms/step - loss: 11019.3076 - val_loss: 7681.1484
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 5554.8237 - val_loss: 3574.3879
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 2602.1719 - val_loss: 1674.9503
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 1233.2937 - val_loss: 795.7920
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 597.2927 - val_loss: 390.3638
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 302.3779 - val_loss: 204.0824
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 165.7404 - val_loss: 118.5876
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 102.2762 - val_loss: 79.6847
Epoch 9/100
19/19 [==============================] - 0s 5ms/step - loss: 72.8303 - val_loss: 62.0338
Epoch 10/100
19/19 [==============================] - 0s 5ms/step -

19/19 [==============================] - 0s 6ms/step - loss: 47.2863 - val_loss: 48.7618
Epoch 82/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2791 - val_loss: 48.7584
Epoch 83/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2814 - val_loss: 48.7683
Epoch 84/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2760 - val_loss: 48.7648
Epoch 85/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2793 - val_loss: 48.7768
Epoch 86/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2794 - val_loss: 48.7702
Epoch 87/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2786 - val_loss: 48.7740
Epoch 88/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2761 - val_loss: 48.7676
Epoch 89/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2789 - val_loss: 48.7671
Epoch 90/100
19/19 [==============================] - 0s 6ms/step - loss: 47.2721 - val_loss: 4

In [11]:
y_pred = model.predict(X_test)

10/10 [==============================] - 0s 3ms/step


In [12]:
print(f'''
The mean square error is {np.round(mean_squared_error(y_test,y_pred),4)}
The root mean square error is {np.round(mean_squared_error(y_test,y_pred, squared = False),4)}
The mean absolute error is {np.round(mean_absolute_error(y_test,y_pred),4)}
''')


The mean square error is 48.7497
The root mean square error is 6.9821
The mean absolute error is 5.8242



#### 6. Can you improve the model performance of question 5 by adjusting the number of neurons or the optimization algorithm?
Yes the model can be improved by adjusting the number of neurons or the optimization algorithm. In the above model the optimization algorithm SGD (Gradient Descent) improved model performance.

#### 7. Build a deep learning regression model to forecast "Scaled sound pressure level" using all other features and PyTorch

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [14]:
X_train = torch.tensor(X_train.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32).reshape(-1,1))

In [15]:
y_train.shape

torch.Size([1202, 1])

In [16]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
#specify the number of neuron for the first hidden layer
hidden_size = 15
print(input_size)
print(output_size)

5
1


In [17]:
# define the child module class derivated from parent class of torch.nn.Module)
class LinearRegressionModel(torch.nn.Module):
    #define the constructor
    def __init__(self, input_size, hidden_size, output_size):
        super(LinearRegressionModel, self).__init__()
        self.hidden = torch.nn.Linear(input_size, hidden_size)  
        self.predict = torch.nn.Linear(hidden_size, output_size)  
    #overife the forward function in this child class
    def forward(self, x):
        x = F.relu(self.hidden(x))     
        y_pred = self.predict(x)            
        return y_pred

In [18]:
model2 = LinearRegressionModel(input_size, hidden_size, output_size)

In [19]:
l = nn.MSELoss()

In [20]:
optimizer = torch.optim.Adam(model2.parameters(), lr=0.05)

C:\Users\MRT\anaconda3\lib\site-packages\torch\cuda\__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [21]:
#fix the random seeds for troch and np
torch.manual_seed(1)
np.random.seed(0)

#set the number of epochs
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass
    y_pred = model2(X_train.requires_grad_())

    #calculate the loss
    loss= l(y_pred, y_train)
    #Set the gradients to be zero
    optimizer.zero_grad()
    
    #backward pass: calculate gradients
    loss.backward()

    #update the weights
    optimizer.step()
  
    print('epoch {0}, loss:{1:.4f}'.format(epoch, loss.item()))

epoch 0, loss:15628.7451
epoch 1, loss:15556.9854
epoch 2, loss:15486.8320
epoch 3, loss:15411.6035
epoch 4, loss:15327.2979
epoch 5, loss:15232.6738
epoch 6, loss:15126.6387
epoch 7, loss:15007.9453
epoch 8, loss:14875.5986
epoch 9, loss:14728.8252
epoch 10, loss:14567.0254
epoch 11, loss:14389.8398
epoch 12, loss:14196.9746
epoch 13, loss:13988.1064
epoch 14, loss:13762.9238
epoch 15, loss:13521.1436
epoch 16, loss:13262.5654
epoch 17, loss:12987.0898
epoch 18, loss:12694.7422
epoch 19, loss:12385.6543
epoch 20, loss:12060.0762
epoch 21, loss:11718.3740
epoch 22, loss:11361.0195
epoch 23, loss:10988.6133
epoch 24, loss:10601.8682
epoch 25, loss:10201.6250
epoch 26, loss:9788.8535
epoch 27, loss:9364.6494
epoch 28, loss:8930.2471
epoch 29, loss:8487.0098
epoch 30, loss:8036.4326
epoch 31, loss:7580.1445
epoch 32, loss:7119.8970
epoch 33, loss:6657.5649
epoch 34, loss:6195.1382
epoch 35, loss:5734.7090
epoch 36, loss:5278.4590
epoch 37, loss:4828.6499
epoch 38, loss:4387.5962
epoch 39,

In [22]:
#convert numpy to tensor
X_test = torch.from_numpy(X_test.astype(np.float32))
#Stop tracking the gradient by calling detach since we don't use it anymore
y_pred2 = model2(X_test).detach().numpy()

In [23]:
print(f'''
The mean square error is {np.round(mean_squared_error(y_test,y_pred2),4)}
The root mean square error is {np.round(mean_squared_error(y_test,y_pred2, squared = False),4)}
The mean absolute error is {np.round(mean_absolute_error(y_test,y_pred2),4)}
''')


The mean square error is 59.3436
The root mean square error is 7.7035
The mean absolute error is 6.2655



#### 8. Can you improve the model performance of question 7 by adjusting the number of neurons or the optimization algorithm?
Yes the model can be improved by changing the number of neurons or optimization algorithm.